In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
from bs4 import BeautifulSoup
import pandas as pd
import time
import re  # Pour extraire la latitude et longitude

browser = webdriver.Chrome()
max_establishment = 15  # Limiter à 10 restaurants

def Selenium_extractor(e, record, max_establishment,filename):
    action = ActionChains(browser)
    time.sleep(5)
    
    
    scroll_count = 0  # Initialiser un compteur
    a = browser.find_elements(By.CLASS_NAME, "hfpxzc")  # Re-fetch elements
    element = a[0]
    while scroll_count < 3:  # Limiter à 3 défilements
        scroll_origin = ScrollOrigin.from_element(element)
        action.scroll_from_origin(scroll_origin, 0, 900).perform()
        time.sleep(2)  # Pause pour laisser le contenu se charger
        scroll_count += 1  # Incrémenter le compteur après chaque défilement


    browser.execute_script("window.scrollTo(0, 0);")
    time.sleep(2)
    # Trouver tous les établissemeent sur la page
    a = browser.find_elements(By.CLASS_NAME, "hfpxzc")
    print(f"Found {len(a)} establishments on the page.")

    # Limiter à 10 restaurants
    for i in range(min(len(a), max_establishment)):
        try:
            a = browser.find_elements(By.CLASS_NAME, "hfpxzc")  # Re-fetch elements
            element = a[i]

            scroll_origin = ScrollOrigin.from_element(element)
            action.scroll_from_origin(scroll_origin, 0, 100).perform()
            action.move_to_element(element).perform()
            element.click()
            time.sleep(2)

            source = browser.page_source
            soup = BeautifulSoup(source, 'html.parser')

            # Extraire les informations principales (nom, téléphone, adresse, site web)
            name_html = soup.find('h1', {"class": "DUwDvf lfPIob"})
            name = name_html.get_text(strip=True) if name_html else "Unknown"

            b = []
            if name not in b:
                e.append(name)
                address = soup.find('div',class_= 'rogA2c').text

                # Extraire la latitude et la longitude depuis l'URL
                coordinates_match = re.search(r'@(-?\d+\.\d+),(-?\d+\.\d+)', browser.current_url)
                if coordinates_match:
                    latitude = coordinates_match.group(1)
                    longitude = coordinates_match.group(2)
                else:
                    latitude = "Not available"
                    longitude = "Not available"

                # Cliquer sur le bouton "Avis Google" (si disponible)
                try:
                    reviews_button = browser.find_element(By.XPATH, "//button[contains(@aria-label, 'Avis')]")
                    reviews_button.click()
                    time.sleep(5)
                    
                    # Faire défiler les avis
                    reviews = []
                    for _ in range(5):  # Ajuster la plage pour plus de défilement
                        action.scroll_by_amount(0, 500).perform()
                        time.sleep(1)
                    
                    review_source = browser.page_source
                    review_soup = BeautifulSoup(review_source, 'html.parser')
                    
                    # Extraire les avis
                    reviews_html = review_soup.findAll('div', {"class": "jftiEf fontBodyMedium"})

                    total_review_element = browser.find_element(By.CSS_SELECTOR, "div.fontBodySmall[jslog]")
                    total_review = total_review_element.text  # Extrait tout le texte (ex : "3 773 avis")
                    total_review = total_review.replace(" avis", "")
                    total_review = total_review.replace("\u202f", "")
                    
                    note =review_soup.find('div', {"class": "fontDisplayLarge"}).text
                    
                    for review in reviews_html:
                        try:
                            reviewer = review.find('div', {"class": "d4r55"}).text
                            review_date = review.find('span', {"class": "rsqaWe"}).text
                            review_text = review.find('span', {"class": "wiI7pd"}).text
                            review_rating = review.find('span', {"class": "kvMYJc"})['aria-label']
                            reviews.append((reviewer, review_date, review_rating, review_text))
                        except Exception as ex:
                            print(f"Error extracting review: {ex}")
                            continue
                except Exception as ex:
                    print(f"Error clicking reviews: {ex}")
                    reviews = [("No reviews", "N/A", "N/A")]

                print([name, address, latitude, longitude,note,total_review, reviews])
                record.append((name, address, latitude, longitude,note, total_review, reviews))
                df = pd.DataFrame(record, columns=['Name', 'Address', 'Latitude', 'Longitude', 'Note','Total_review', 'Reviews'])
                df.to_csv(filename + '.csv', index=False, encoding='utf-8')

            # Limiter à 10 restaurants
            if len(record) >= max_establishment:
                break
        except Exception as ex:
            print(f"Error: {ex}")
            continue



For this first part we will focus on museums

In [17]:
filename = "museum_with_reviews"
link = "https://www.google.com/maps/search/mus%C3%A9e/@48.8674214,2.2826524,12z/data=!3m1!4b1?entry=ttu&g_ep=EgoyMDI0MTIxMS4wIKXMDSoASAFQAw%3D%3D"
record = []
e = []
browser.get(str(link))
time.sleep(10)
Selenium_extractor(e, record, max_establishment,filename)


Found 14 establishments on the page.
['Musée de Montmartre', '12 Rue Cortot, 75018 Paris', '48.8673627', '2.2826522', '4,5', '3756', [('antoine delforge', 'il y a une semaine', '5\xa0étoiles', 'Super musée.\nRafraîchissant, étonnant, avec une vue à couper le souffle.\nL’accueil est adorable. …'), ('Cécile L.', 'il y a 3\xa0mois', '5\xa0étoiles', "Super musée ! J'ai adoré la collection permanente comme la temporaire. Tout à fait faisable avec des enfants si on prend le temps de leur expliquer un peu. Les jardins sont magnifiques et le petit café est bien agréable et pas trop cher.\nJe recommande."), ('Let me no', 'il y a 5\xa0mois', '5\xa0étoiles', "J'ai beaucoup aimé cette visite. Le musée est riche en toiles, affiches...l'exposition très instructive. Le lieu est magique, le jardin très agréable. Une bulle d'oxygène en plein Paris"), ('Philippe Morin', 'il y a un mois', '5\xa0étoiles', 'J’adore ce petit musée avec son petit jardin calme au cœur de Montmartre (café et terrasse), sa vue 

Now on parks 

In [18]:
filename = "parks_with_reviews"
link = "https://www.google.com/maps/search/parc/@48.8675298,2.2826521,12z/data=!3m1!4b1?entry=ttu&g_ep=EgoyMDI0MTIxMS4wIKXMDSoASAFQAw%3D%3D"
record = []
e = []
browser.get(str(link))
time.sleep(10)
Selenium_extractor(e, record, max_establishment,filename)

Found 14 establishments on the page.
['Parc de Billancourt', '32 All. Robert Doisneau, 92100 Boulogne-Billancourt', '48.8262705', '2.0836272', '4,4', '1355', [('Frédéric G', 'il y a 9\xa0mois', '4\xa0étoiles', 'Très bel endroit, surtout en cette période de début Mars, avec les cerisiers en fleurs. Seul bémol, parc interdit aux chiens, eux aussi ont le droit de se promener'), ('cdgbfamily', 'il y a 9\xa0mois', '5\xa0étoiles', 'Un parc magnifique où il fait bon se promener tout au long de l’année mais surtout au mois de mars avec les cerisiers en fleurs.\nUn parc qui égaie le quartier'), ('Marina', 'il y a 3\xa0ans', '5\xa0étoiles', "Petite mise à jour: penser à aller dans ce parc mi-mars pour voir les arbres en fleurs !!!! Excellente surprise dans un univers bétonné. Il y a beaucoup de bancs et sièges pour apprécier cet endroit. Le petit plan d'eau et ses canards ... …"), ('mana k', 'il y a un an', '5\xa0étoiles', 'Le plus bel endroit du quartier du Trapèze. Un écrin de verdure avec 2 e

Then Art galleries

In [19]:
filename = "art_with_reviews"
link = "https://www.google.com/maps/search/galeries+d'art/@48.867697,2.282652,12z/data=!3m1!4b1?entry=ttu&g_ep=EgoyMDI0MTIxMS4wIKXMDSoASAFQAw%3D%3D"
record = []
e = []
browser.get(str(link))
time.sleep(10)
Selenium_extractor(e, record, max_establishment,filename)

Found 14 establishments on the page.
["Galerie d'art Carré d'artistes Saint-Germain", '66 Rue Saint-André des Arts, 75006 Paris', '48.8676383', '2.2826518', '4,8', '41', [('JC 75', 'il y a 3\xa0mois', '5\xa0étoiles', "Très belle galerie avec de très belles toiles de différents artistes aux talents uniques. Un très grand choix pour se faire plaisir. Accueil très sympathique, politesse et sourire sont au rendez-vous. Au plaisir d'y retourner..."), ("Edouard Puech Pays d'Alissac", 'il y a 8\xa0mois', '5\xa0étoiles', 'Accueil charmant de la part de l’équipe , spécialement Renato et Audrey. Un plaisir d’y être guidé dans la découverte des œuvres et artistes. Au plaisir d’y revoir du Julien Rey.'), ('Martin Lavergne', 'il y a 10\xa0mois', '5\xa0étoiles', "Ravi d'avoir exposé à 2 reprises dans cette jolie galerie et même d'avoir eu l'occasion de faire une performance live avec un accrochage de grands formats ! Super équipe ! Moogly"), ('Corinne Malfreyt', 'il y a 10\xa0mois', '5\xa0étoiles', 

Now mall

In [20]:
filename = "mall_with_reviews"
link = "https://www.google.com/maps/search/centre+commercial/@48.8678641,2.2826519,12z/data=!3m1!4b1?entry=ttu&g_ep=EgoyMDI0MTIxMS4wIKXMDSoASAFQAw%3D%3D"
record = []
e = []
browser.get(str(link))
time.sleep(10)
Selenium_extractor(e, record, max_establishment,filename)

Found 14 establishments on the page.
['Centre Commercial Italie Deux', "30 Av. d'Italie, 75013 Paris", '48.8678054', '2.2826517', '4,0', '16906', [('Sofia Gomes', 'il y a 5\xa0jours', '3\xa0étoiles', 'Le centre commercial Italie deux est un bon centre commercial accueillant rempli de bon choix de boutique mais le seule inconvénient est qu’ils savent pas tenir les magasins à long terme ! Ce que je veux dire c’est ce que en ce moment il a beaucoup de magasins qui font leurs ouverture dans ce centre commercial mais pour un petit terme.'), ('Jacobo GARCIA', 'il y a un mois', '2\xa0étoiles', 'La température dans ce centre c’est juste intenable. Il faisait 13•c dehors mais à l’intérieur on était plus proche du 25. Même les employés des boutiques étaient en tee-shirts et avaient chaud. On a été obligé de partir au bout d’une …'), ('Pierre Renaud', 'il y a un mois', '5\xa0étoiles', "Le centre commercial Porte d'Italie est plutôt sympa et pratique ! On y trouve un bon mix de boutiques en tout g

Now bike rental

In [21]:
filename = "bike_with_reviews"
link = "https://www.google.com/maps/search/location+v%C3%A9lo/@48.8681983,2.2826517,12z/data=!3m1!4b1?entry=ttu&g_ep=EgoyMDI0MTIxMS4wIKXMDSoASAFQAw%3D%3D"
record = []
e = []
browser.get(str(link))
time.sleep(10)
Selenium_extractor(e, record, max_establishment,filename)

Found 14 establishments on the page.
['6clo', '36 Rue Saint-Paul, 75004 Paris', '48.8681396', '2.2826515', '4,9', '363', [('Joel Ronez', 'il y a 7\xa0mois', '5\xa0étoiles', "Super équipement (Sirrus), super concept, ouvert tous les jours. Vélos en parfait état, remise ponctuelle, et dépannage avec le café à côté en dehors des heures d'ouverture. Tarifs vraiment top. Aller à Nemours depuis Paris en suivant la Seine (130 km) en 2 jours), et vélo très adapté pour les différents revêtements."), ('Flora Magno', 'il y a 3\xa0mois', '5\xa0étoiles', 'Service de location pratique et efficace! On est arrivé à 10h et Bokolo (très sympa) nous a très bien reçu et préparé le vélo rapidement. Équipement en parfait état! …'), ('Martini Thibaut', 'il y a 3\xa0mois', '4\xa0étoiles', "Bonne expérience avec 6clo. J'ai loué 2 vélos pour 5 jours, pas de mauvaise surprise ou de surcoûts cachés. Les vélos sont très confortables et de qualité. Petit bémol, nous avons eu à faire un peu de mécanique sur le bord 

guided tour

In [22]:
filename = "tour_with_reviews"
link = "https://www.google.com/maps/search/visite+guid%C3%A9e/@48.8683655,2.2826516,12z/data=!3m1!4b1?entry=ttu&g_ep=EgoyMDI0MTIxMS4wIKXMDSoASAFQAw%3D%3D"
record = []
e = []
browser.get(str(link))
time.sleep(10)
Selenium_extractor(e, record, max_establishment,filename)

Found 14 establishments on the page.
Error extracting review: 'NoneType' object has no attribute 'text'
['Paris Visites Guidées', 'Paris Visites - Michel Faul, 92 Rue Marius Aufan, 92300 Levallois-Perret', '48.8683068', '2.2826514', '5,0', '5', [('Dominique Pint', 'il y a 2\xa0ans', '5\xa0étoiles', 'Très belle visite du quartier du Parc Monceau avec un guide passionnant et passionné. De nombreuses anecdotes émaillent son intervention et il sait tenir en haleine les participants. Nous le recommandons vivement.'), ('B PERDRIAU', 'il y a un an', '5\xa0étoiles', "Super parcours,  guide très intéressant avec une petite d'humour.\nMerci François."), ('Jean Marc Lefevre', 'il y a 7\xa0ans', '5\xa0étoiles', 'Bien organisé'), ('Sylvia GREGORY', 'il y a 3\xa0mois', '5\xa0étoiles', "Mon mari et moi avons récemment eu un guide du Louvre.  Olivia Putorti a expliqué l'arrière-plan de nombreuses peintures et sculptures d'une manière informative et intéressante.  C'est une guide charmante et merveille

cruise

In [23]:
filename = "cruise_with_reviews"
link = "https://www.google.com/maps/search/croisi%C3%A8re/@48.8685326,2.2826515,12z/data=!3m1!4b1?entry=ttu&g_ep=EgoyMDI0MTIxMS4wIKXMDSoASAFQAw%3D%3D"
record = []
e = []
browser.get(str(link))
time.sleep(10)
Selenium_extractor(e, record, max_establishment,filename)

Found 14 establishments on the page.
['Paris Seine', 'at the foot of the Orsay Museum, Port Solferino, 75007 Paris', '48.8684739', '2.2826513', '4,4', '1605', [('Ayour EL', 'il y a 3\xa0semaines', '5\xa0étoiles', 'Nous avons passé un excellent déjeuner à bord du bateau-restaurant. Tout était parfait ! Le service, assuré par Georges, était exceptionnel, avec une attention particulière portée à chaque détail. Il a su être à la fois professionnel et …'), ('Kahaia Raparii', 'il y a un mois', '4\xa0étoiles', 'À faire au moins une fois dans sa vie .\nVoir même à refaire c’était top en famille et entre amis 😊✨\nLe service bien… …'), ('Jean Marc Alves', 'il y a un mois', '5\xa0étoiles', 'Invité pour mes 40 ans par ma femme, j’ai apprécié le cadre et la cuisine. Le service quand à lui était impeccable, Céline était très souriante et avenante. Elle a contribué largement à notre bonne opinion sur le restaurant. …'), ('Charlie Canova', 'il y a 2\xa0mois', '5\xa0étoiles', '800km pour le concert au 

https://open-meteo.com/en/docs/historical-weather-api#hourly=temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,snowfall,snow_depth,weather_code,surface_pressure,cloud_cover,wind_speed_10m,wind_speed_100m,wind_direction_10m,soil_temperature_100_to_255cm,soil_moisture_100_to_255cm&daily=temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,sunrise,sunset,daylight_duration,sunshine_duration

In [26]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 48.86,
	"longitude": 2.28,
	"start_date": "2014-12-10",
	"end_date": "2024-12-10",
	"daily": ["weather_code", "temperature_2m_max", "temperature_2m_min", "temperature_2m_mean", "apparent_temperature_max", "apparent_temperature_min", "apparent_temperature_mean", "sunrise", "sunset", "daylight_duration", "sunshine_duration", "precipitation_sum", "rain_sum", "snowfall_sum", "precipitation_hours", "wind_speed_10m_max", "wind_gusts_10m_max", "wind_direction_10m_dominant", "shortwave_radiation_sum"],
	"timezone": "Europe/Berlin"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()
daily_temperature_2m_mean = daily.Variables(3).ValuesAsNumpy()
daily_apparent_temperature_max = daily.Variables(4).ValuesAsNumpy()
daily_apparent_temperature_min = daily.Variables(5).ValuesAsNumpy()
daily_apparent_temperature_mean = daily.Variables(6).ValuesAsNumpy()
daily_sunrise = daily.Variables(7).ValuesAsNumpy()
daily_sunset = daily.Variables(8).ValuesAsNumpy()
daily_daylight_duration = daily.Variables(9).ValuesAsNumpy()
daily_sunshine_duration = daily.Variables(10).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(11).ValuesAsNumpy()
daily_rain_sum = daily.Variables(12).ValuesAsNumpy()
daily_snowfall_sum = daily.Variables(13).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(14).ValuesAsNumpy()
daily_wind_speed_10m_max = daily.Variables(15).ValuesAsNumpy()
daily_wind_gusts_10m_max = daily.Variables(16).ValuesAsNumpy()
daily_wind_direction_10m_dominant = daily.Variables(17).ValuesAsNumpy()
daily_shortwave_radiation_sum = daily.Variables(18).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["weather_code"] = daily_weather_code
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["apparent_temperature_max"] = daily_apparent_temperature_max
daily_data["apparent_temperature_min"] = daily_apparent_temperature_min
daily_data["apparent_temperature_mean"] = daily_apparent_temperature_mean
daily_data["sunrise"] = daily_sunrise
daily_data["sunset"] = daily_sunset
daily_data["daylight_duration"] = daily_daylight_duration
daily_data["sunshine_duration"] = daily_sunshine_duration
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["rain_sum"] = daily_rain_sum
daily_data["snowfall_sum"] = daily_snowfall_sum
daily_data["precipitation_hours"] = daily_precipitation_hours
daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
daily_data["wind_gusts_10m_max"] = daily_wind_gusts_10m_max
daily_data["wind_direction_10m_dominant"] = daily_wind_direction_10m_dominant
daily_data["shortwave_radiation_sum"] = daily_shortwave_radiation_sum

daily_dataframe = pd.DataFrame(data = daily_data)
daily_dataframe.to_csv("daily_data.csv", index=False, encoding="utf-8")

print("DataFrame saved as 'daily_data.csv'.")
print(daily_dataframe)


Coordinates 48.892791748046875°N 2.292020559310913°E
Elevation 72.0 m asl
Timezone b'Europe/Berlin' b'CET'
Timezone difference to GMT+0 3600 s
DataFrame saved as 'daily_data.csv'.
                          date  weather_code  temperature_2m_max  \
0    2014-12-09 23:00:00+00:00          53.0            9.615499   
1    2014-12-10 23:00:00+00:00          53.0            8.915500   
2    2014-12-11 23:00:00+00:00          63.0           11.115499   
3    2014-12-12 23:00:00+00:00          61.0            7.165500   
4    2014-12-13 23:00:00+00:00           3.0            2.965500   
...                        ...           ...                 ...   
3649 2024-12-05 23:00:00+00:00          53.0           13.365499   
3650 2024-12-06 23:00:00+00:00          61.0           11.665500   
3651 2024-12-07 23:00:00+00:00          61.0            8.265500   
3652 2024-12-08 23:00:00+00:00          53.0            6.365500   
3653 2024-12-09 23:00:00+00:00          51.0            6.015500   

   

In [27]:
import os

csv_files = [
    "art_with_reviews.csv",
    "bike_with_reviews.csv",
    "cruise_with_reviews.csv",
    "mall_with_reviews.csv",
    "parks_with_reviews.csv",
    "tour_with_reviews.csv",
    "museum_with_reviews.csv"
]

categories = {
    "art_with_reviews.csv" : "art",
    "bike_with_reviews.csv": "bike",
    "cruise_with_reviews.csv":"cruie",
    "mall_with_reviews.csv": "mall",
    "parks_with_reviews.csv": "parks",
    "tour_with_reviews.csv": "tour",
    "museum_with_reviews.csv": "museum"
}

dataframes = []

for file in csv_files:
    if os.path.exists(file):  # Vérifie si le fichier existe
        # Charger le fichier CSV
        df = pd.read_csv(file)
        # Ajouter une colonne 'Category' avec la catégorie extraite
        df['Category'] = categories[file]
        dataframes.append(df)
    else:
        print(f"File not found: {file}")

# Combiner tous les DataFrames en un seul
combined_df = pd.concat(dataframes, ignore_index=True)

# Enregistrer le DataFrame combiné en un fichier CSV
combined_df.to_csv("combined_with_reviews.csv", index=False, encoding="utf-8")
print("Combined dataset saved as 'combined_with_reviews.csv'.")


Combined dataset saved as 'combined_with_reviews.csv'.
